In [24]:
import numpy as np
import pandas as pd

In [25]:
# Use this because I couldn't see enough output
pd.set_option('display.max_rows', None)

In [26]:
df = pd.read_csv('export_format.csv')

In [27]:
df.head()

,employer,period_begin,period_end,post_date,salary,sheltered,unsheltered,total
0,East Baton Rouge,8/1/1988,8/31/1988,6/30/1989,223.25,0.0,20.09,20.09
1,East Baton Rouge,9/1/1988,9/30/1988,6/30/1989,532.65,0.0,47.94,47.94
2,East Baton Rouge,10/1/1988,10/31/1988,6/30/1989,407.50,0.0,36.68,36.68
3,East Baton Rouge,11/1/1988,11/30/1988,6/30/1989,407.50,0.0,36.68,36.68
4,East Baton Rouge,12/1/1988,12/31/1988,6/30/1989,407.50,0.0,36.68,36.68


In [28]:
# Ran into a problem converting salary - reading export_format.csv worked fine, 
# but reading export_format4.csv did not!
# here i notice that salary is an object, so that means string right?
# how come the export for salary was float64 for exportformat, but not exportformat4???? weird
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 408 entries, 0 to 407
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   employer      394 non-null    object 
 1   period_begin  394 non-null    object 
 2   period_end    394 non-null    object 
 3   post_date     394 non-null    object 
 4   salary        394 non-null    float64
 5   sheltered     394 non-null    float64
 6   unsheltered   394 non-null    float64
 7   total         394 non-null    float64
dtypes: float64(4), object(4)
memory usage: 25.6+ KB


In [31]:
# this is executed on export_format4
df = df.groupby('period_end').sum()
    

In [32]:
df.head()

,salary,sheltered,unsheltered,total
period_end,,,,
1/31/1989,783.66,0.0,70.54,70.54
1/31/1990,865.40,0.0,60.57,60.57
1/31/1991,936.56,0.0,65.56,65.56
1/31/1992,974.04,0.0,68.18,68.18
1/31/1993,1013.48,0.0,70.96,70.96


In [36]:
# So, at first I ran all these on 'export_format.csv'
# this cell was ran on export_format.csv, and it 'worked' (but the output did not match my expected result)
# when I applied the above on export_format.csv, i noticed my df did not sort appropriately
# before, the df was sorting by months instead of true linear dates
df = df.sort_values(by='period_end', key=pd.to_datetime)

In [37]:
df.head()

,salary,sheltered,unsheltered,total,rolling_60
period_end,,,,,
8/31/1988,317.29,0.0,28.55,28.55,NaN
9/30/1988,908.81,0.0,81.80,81.80,NaN
10/31/1988,783.66,0.0,70.54,70.54,NaN
11/30/1988,783.66,0.0,70.54,70.54,NaN
12/31/1988,971.74,0.0,87.47,87.47,NaN


In [38]:
#note that this did work on df = export_format but it did not work on df = export_format4 (because 'salary' column is not an int)
df['rolling_60'] = df['salary'].rolling(60).mean()

In [39]:

# the rolling_60 values did NOT match my expected results (see file samp_salary). expected results also listed 'export_key'
df.tail()

,salary,sheltered,unsheltered,total,rolling_60
period_end,,,,,
3/31/2022,8549.38,598.45,0.0,598.45,7195.089833
4/30/2022,8652.05,605.64,0.0,605.64,7229.351333
5/31/2022,8898.70,622.90,0.0,622.90,7283.091833
6/30/2022,7237.25,506.60,0.0,506.60,7307.999667
7/31/2022,10013.83,700.96,0.0,700.96,7347.837000


In [40]:
# this was executed on export_format
df['salary'].info()

<class 'pandas.core.series.Series'>
Index: 179 entries, 8/31/1988 to 7/31/2022
Series name: salary
Non-Null Count  Dtype  
--------------  -----  
179 non-null    float64
dtypes: float64(1)
memory usage: 2.8+ KB


In [113]:
def salary_cap(rolling_60):
    for i in len(rolling_60):
        for num[i] in rolling_60:
            # well............actually this isn't going to work at all BECAUSE
            # i need this to happen within the .rolling() function, not AFTER
            # if called on the rolling_60 column, then i'm just implementing the cap end result, not within the year to year

SyntaxError: invalid syntax (2300947213.py, line 1)

In [46]:
#i wrote this function before i discovered pd.to_numeric
#....it didn't work, could you tell me why?

def int_convert(notnum):
    convert = []
    for num in notnum:
        convert += int(num)
    return convert